In [79]:
# script to convert FLUKA energy deposition to heat
import numpy as np
import pandas as pd
from decimal import Decimal

In [100]:
def getFiles(inp):
    cycles = 5
    start = r"1Region"
    end = r"Total (integrated over volume):"
    Energy = []
    Region = []
    data = []
    for i in range(1, cycles+1):
        file = inp + f'00{i}.out'
        try:
            lines = []
            with open(file, 'r') as f:
                appending = False
                for line in f:
                    if end in line:
                        break
                    if start in line:
                        appending = True
                    if appending is True:
                        lines.append(line.split())
                lines = lines[4:-1]
            data.append(lines)

        except FileNotFoundError:
            print('File not found')

    for line in data:
        for row in line:
            Region.append(row[1])
            erg = float(row[5].replace('D', 'E'))
            Energy.append(erg)

    dict = {'Region': Region, 'Energy': Energy}
    df = pd.DataFrame(dict)

    return df


def getInfo(inp, vol=1000.0, current=0.006):
    df = getFiles(inp)
    regions = df['Region'].unique().tolist()
    print(r'''REGION   :: Heat (W)   +/- Standard Deviation
----------------------------------''')
    for region in regions:
        values = df['Energy'].where(df['Region'] == region)
        mean = erg2heat(values.mean(), vol, current=current)
        std = erg2heat(values.std(), vol, current=current)
        print(f'{region:8} :: {mean:.4e} +/- {std:.4e}')


def erg2heat(val, vol, current=0.006):
    '''energy dep * volume * 1.6E-10 [to convert to joules] *
    current / charge [equivalent to primaries (real) per time]
    == heat (watts)'''
    heat = val * 1E9 * current * vol
    return heat


inp = 'linac_sys_target'
getInfo(inp)


REGION   :: Heat (W)   +/- Standard Deviation
----------------------------------
BLKBODY  :: 1.4401e+02 +/- 1.5012e-01
VOID     :: 0.0000e+00 +/- 0.0000e+00
WATER    :: 9.7696e+02 +/- 5.4334e+01
BASE     :: 5.7315e+06 +/- 1.2358e+03
SUB      :: 6.7866e+06 +/- 1.1351e+03
TARGET   :: 2.4809e+06 +/- 1.7268e+02
BLANK    :: 0.0000e+00 +/- 0.0000e+00
CAN1     :: 3.0636e+00 +/- 4.2581e-02
WATERCNT :: 7.4887e+01 +/- 1.4430e-01
RFTR     :: 1.0004e+03 +/- 2.4529e+01
CANVOID  :: 0.0000e+00 +/- 0.0000e+00
CAN2     :: 8.7359e+02 +/- 7.4283e+01
VOIDCONE :: 0.0000e+00 +/- 0.0000e+00
DET100   :: 0.0000e+00 +/- 0.0000e+00
DET200   :: 0.0000e+00 +/- 0.0000e+00
DET300   :: 0.0000e+00 +/- 0.0000e+00
DET400   :: 0.0000e+00 +/- 0.0000e+00
DET500   :: 0.0000e+00 +/- 0.0000e+00
DETOUT   :: 0.0000e+00 +/- 0.0000e+00
